In [10]:
import paramiko 
import pandas as pd
import json
from datetime import datetime
from libs.utils import *
from libs.fetcher import *

## Set constants

In [11]:
mod_var = "ICU"

## Download UMCPHR data from SFTP

In [12]:
pull_fname = "mt_region_hosp_icu_vent.csv"
remotepath = "/celFtpFiles/covid19/modcomp/outgoing/" + pull_fname
localpath = "data/umcphr/" + pull_fname
ftp_get(remotepath, localpath)

## Bring in data from model simulations

In [13]:
if mod_var == 'Hospitalizations':
    umcphr_fname = 'hospitalizations'
    ihme = IhmeData().get_allbed()
    can = CanData().get_allbed(scenarios=[1, 3])
    chime = ChimeData().get_allbed()
    umcphr = UmCphrData().get_allbed()
    mod_list = [ihme, can, chime, umcphr]
    mod_list = [ihme, can, chime]
elif mod_var == 'ICU':
    umcphr_fname = 'icu'
    ihme = IhmeData().get_icubed()
    chime = ChimeData().get_icubed()
    umcphr = UmCphrData().get_icubed()
    mod_list = [ihme, chime, umcphr]

df = pd.DataFrame()
for mod in mod_list:
    df = df.merge(
        mod,
        how='outer',
        left_index=True,
        right_index=True
        )

df = df.interpolate(method='polynomial', order=3)
df.bfill(0, inplace=True)
df = df[(df.index >= '2020-03-10') & (df.index <= '2020-07-10')]

## Write to csv

In [14]:
date = datetime.today().strftime("%m%d%Y") 
push_fname = 'data/modcom_{}_{}.csv'.format(umcphr_fname, date)
df.to_csv(push_fname)

## Push to FTP

In [15]:
remotepath = "/celFtpFiles/covid19/modcomp/incoming/"
filesoutdir = "/home/nick/workspace/cov19-mod-comp/data/"
filenameFTP = push_fname.split("/")[-1]
localpath = filesoutdir + filenameFTP
ftp_push(localpath, remotepath)
